# NNMD of zeolites using the pretrained models

In this notebook, we perform a NN-based MD NVE simulation using ASE and the [SchNet NN potential](https://github.com/learningmatter-mit/NeuralForceField). We will be using the second generation of zeolite models, as shown in [our paper](https://arxiv.org/abs/2101.11588).

The utilities at `nff` will be used to perform the MD simulation. `nglview` will be used to visualize the generated trajectories.

In [1]:
import os
import random
import numpy as np
from ase import Atoms, units
from ase.md.verlet import VelocityVerlet

from nff.io import NeuralFF, AtomsBatch, EnsembleNFF
from nff.md.nve import Dynamics
from nff.data import Dataset
import nff.utils.constants as const

import nglview as nv
from ase.io import Trajectory, read

## Loading the models and parameters

The dataset contains a PyTorch dataset with all the DFT data used to train the third generation of NN potentials. Here, we will use the pre-trained ensemble. For learning how to train the models using the SchNet architecture and the current dataset, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [2]:
dset = Dataset.from_file('../data/zeolite.pth.tar')

`DEVICE` sets the GPU used for evaluating the model. If you want to evaluate on a CPU, set `DEVICE = 'cpu'`. The models are stored at `/models/zeolite`, where `/` is the root folder of this repository.

In [3]:
DEVICE = 0

def get_ensemble_calc(device=DEVICE):
    PATH = '../models/zeolite'
    models = []
    for model_name in sorted(os.listdir(PATH)):
        m = NeuralFF.from_file(os.path.join(PATH, model_name), device=device).model
        models.append(m)

    return EnsembleNFF(models, device=device)

ensemble = get_ensemble_calc(device=DEVICE)

Here, we set the parameters for the MD simulation. For learning how to use these parameters within our code, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [4]:
def get_md_params(traj_filename, temperature=1000):
    return {
        'T_init': temperature,
        'time_step': 0.5,
        'thermostat': VelocityVerlet,  
        'thermostat_params': {'timestep': 0.5 * units.fs},
        'steps': 2 * 2000,
        'save_frequency': 40,
        'nbr_list_update_freq': 5,
        'thermo_filename': 'thermo.log',
        'traj_filename': traj_filename,
        'skip': 0
    }

Finally, we use the lowest energy conformation within the existing dataset (i.e. the ground state of ammonia) as a starting configuration for the MD simulation. `AtomsBatch` is a [wrapper within our NFF repo](https://github.com/learningmatter-mit/NeuralForceField/blob/master/nff/io/ase.py) and can be used to interface an ASE atom with NFF.

In [5]:
CUTOFF = 5.0

def get_md_atoms(dset=dset, cutoff=CUTOFF, device=DEVICE):
    props = random.choice(dset)

    atoms = AtomsBatch(
        positions=props['nxyz'][:, 1:],
        numbers=props['nxyz'][:, 0],
        cell=props['lattice'],
        pbc=True,
        cutoff=CUTOFF,
        props={'energy': 0, 'energy_grad': []},
        calculator=ensemble,
        device=device,
    )
    _ = atoms.update_nbr_list()
    
    return atoms

## Performing the MD simulation

Now, we perform the MD simulation using the parameters shown before.

In [6]:
TEMPERATURE = 1000

name = f'NVE_{TEMPERATURE}.traj'

atoms = get_md_atoms()
md_params = get_md_params(name, TEMPERATURE)
dyn = Dynamics(atoms, md_params)
traj = dyn.run()

/home/dskoda/.conda/envs/nff/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))
/home/dskoda/.conda/envs/nff/lib/python3.7/site-packages/ase/atoms.py:968: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000          21.0118      14.0108       7.0010   902.7

0.0200          20.8475      18.3280       2.5195   324.9

0.0400          21.0118      17.4975       3.5143   453.1

0.0600          20.9631      17.7914       3.1717   409.0

0.0800          20.9020      16.9618       3.9402   508.0

0.1000          20.9914      17.7971       3.1944   411.9

0.1200          20.8272      17.3013       3.5259   454.6

0.1400          20.7450      17.1150       3.6300   468.1

0.1600          21.0548      17.0991       3.9557   510.0

0.1800          20.8401      17.5986       3.2415   418.0

0.2000          21.1996      17.4275       3.7721   486.4

0.2200          21.1101      17.5906       3.5195   453.8

0.2400          21.0108      17.6698       3.3409   430.8

0.2600          21.0854      17.5215       3.5638   459.5

0.2800          20.8050      17.1830       3.6219   467.0

0.3000          20.7266      17.6601       3.0665   395.4


## Visualizing the trajectory

Now we visualize the generated trajectory. The translation makes it easier to see the dynamics of the host within `nglview`.

In [7]:
filetraj = Trajectory(name)

newtraj = []
for at in filetraj:
    at.translate(at.cell.sum(0) * np.array([0.5, 0.5, 0.5]))
    at.wrap()
    newtraj.append(at)

In [8]:
view = nv.show_asetraj(newtraj)
view.add_unitcell()

view

NGLWidget(max_frame=100)